What is the average number of bike rentals per hour in individual stations?

We still have to create a dependent variable(number of trips per hour for each station ) column in our dataframe. To do that, we can then group by station_id , start_time and hour and then count the number of trips per hour. Then we can simply do a merge on the two dataframes. Let's add a num_trip_hr column as below:

In [ ]:
#import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [295]:
#read the csv file
trip = pd.read_csv("Healthy_rides_data.csv", parse_dates = ["start_time","stop_time"])

In [314]:
trip.head(2)

,start_time,trip_id,stop_time,bike_id,trip_duration,from_station_id,from_station_name,to_station_id,station_name,user_type,...,event_type,rain,wind,weather,hour,month,speed,day_of_the_week,tripdate,num_trips_day
0,2015-05-31 07:18:00,10677999,2015-05-31 07:52:00,70381,2018,1019,42nd St & Butler St,1001,Forbes Ave & Market Square,Subscriber,...,N,0.37,8.95,good,7,5,5.41,Sunday,2015-05-31,158
1,2015-05-31 07:25:00,10678335,2015-05-31 07:57:00,70331,1900,1005,Forbes Ave & Grant St,1000,Liberty Ave & Stanwix St,Customer,...,N,0.37,8.95,good,7,5,0.80,Sunday,2015-05-31,158


In [315]:
#We already have the number of trips per day column(num_trips_day)
trip.sort_values("start_time")
trip.shape

(163377, 28)

In [316]:
# Create a Group by object 
grp_date = trip.groupby(by = [ 'from_station_id','start_time','hour'])

# Compute number of trips by date and check out the result
trips_by_hour = pd.DataFrame(grp_date.size(), columns=['num_trips_hr'])

trips_by_hour.head()

num_trips_hr
from_station_id start_time          hour              
1000            2015-05-31 09:31:00 9                1
                2015-05-31 09:50:00 9                1
                2015-05-31 09:51:00 9                2
                2015-05-31 10:07:00 10               1
                2015-05-31 11:18:00 11               2

In [317]:
trips_by_hour.reset_index(level=[0,2], inplace=True)

trips_by_hour.head()

,from_station_id,hour,num_trips_hr
start_time,,,
2015-05-31 09:31:00,1000,9,1
2015-05-31 09:50:00,1000,9,1
2015-05-31 09:51:00,1000,9,2
2015-05-31 10:07:00,1000,10,1
2015-05-31 11:18:00,1000,11,2


In [318]:
trips_by_hour.reset_index(level=[0], inplace=True)

trips_by_hour.head()

,start_time,from_station_id,hour,num_trips_hr
0,2015-05-31 09:31:00,1000,9,1
1,2015-05-31 09:50:00,1000,9,1
2,2015-05-31 09:51:00,1000,9,2
3,2015-05-31 10:07:00,1000,10,1
4,2015-05-31 11:18:00,1000,11,2


In [319]:
#we dont need these duplicate columns anymore
trips_by_hour.drop(["from_station_id",'hour'], axis = 1,inplace = True)

In [320]:
#Sort date values before merging:

trips_by_hour.sort_values("start_time")

trips_by_hour.shape

(152619, 2)

In [321]:
trips_by_hour.dtypes

start_time      datetime64[ns]
num_trips_hr             int64
dtype: object

In [322]:
#merge two dataframes
df = pd.merge(trip,trips_by_hour , on='start_time')

In [323]:
df['num_trips_hr'].value_counts()

1    171685
2     20070
3      2292
4       439
5        85
6        25
7         7
Name: num_trips_hr, dtype: int64

In [324]:
df.isna().sum()

start_time             0
trip_id                0
stop_time              0
bike_id                0
trip_duration          0
from_station_id        0
from_station_name      0
to_station_id          0
station_name           0
user_type              0
TMAX                  28
TMIN                  28
season                 0
holiday                0
walk_score             0
transit_score          0
bike_score             0
distance               0
event_type             0
rain                   0
wind                 186
weather                0
hour                   0
month                  0
speed                  0
day_of_the_week        0
tripdate               0
num_trips_day          0
num_trips_hr           0
dtype: int64

In [325]:
#Since the start date column were in order, I am filling all the null values using forward fill method
df["TMAX"] = df["TMAX"].fillna(method='ffill')
df["TMIN"] = df["TMIN"].fillna(method='ffill')
df["wind"] = df["wind"].fillna(method='ffill')

In [326]:
df.isna().sum()

start_time           0
trip_id              0
stop_time            0
bike_id              0
trip_duration        0
from_station_id      0
from_station_name    0
to_station_id        0
station_name         0
user_type            0
TMAX                 0
TMIN                 0
season               0
holiday              0
walk_score           0
transit_score        0
bike_score           0
distance             0
event_type           0
rain                 0
wind                 0
weather              0
hour                 0
month                0
speed                0
day_of_the_week      0
tripdate             0
num_trips_day        0
num_trips_hr         0
dtype: int64

In [327]:
df.isnull().sum()

start_time           0
trip_id              0
stop_time            0
bike_id              0
trip_duration        0
from_station_id      0
from_station_name    0
to_station_id        0
station_name         0
user_type            0
TMAX                 0
TMIN                 0
season               0
holiday              0
walk_score           0
transit_score        0
bike_score           0
distance             0
event_type           0
rain                 0
wind                 0
weather              0
hour                 0
month                0
speed                0
day_of_the_week      0
tripdate             0
num_trips_day        0
num_trips_hr         0
dtype: int64

In [328]:
#y= df.to_csv("Rides.csv", index = False)

In [329]:
df.shape

(194603, 29)